# Création d'un jeu de données nettoyé

Avant de faire un prétraitement des données et d'effectuer des classifications, on va chercher à nettoyer les données en retirant les lignes où les textes/titres sont vides, en supprimant les doublons.

In [2]:
import pandas as pd

#Importation des jeux de données
dfInit_train = pd.read_csv('./data_init/HAI817_Projet_train.csv',sep=',')
dfInit_test = pd.read_csv('./data_init/HAI817_Projet_test.csv',sep=',')

In [3]:
#Concaténation des deux jeux de données test et train
dfInit_concat = pd.concat([dfInit_train,dfInit_test], ignore_index = True)

#Suppression des mixture et other
df_concat = dfInit_concat.loc[~dfInit_concat['our rating'].isin(['mixture','other'])]

#Suppression des doublons
df_concat = df_concat.drop_duplicates()

#Suppression des titres NaN
df_concat = df_concat.loc[~df_concat['title'].isin(['NaN'])]

#Suppression des textes NaN
df_concat = df_concat.loc[~df_concat['text'].isin(['NaN'])]

#Concaténation du titre et du texte dans une nouvelle colonne title_text
df_concat['title_text'] = df_concat['title'] +"  "+ df_concat['text']

#Affichage de la taille des données
print("Taille des Data train+test :")
print(df_concat['our rating'].value_counts())

Taille des Data train+test :
false    815
true     411
Name: our rating, dtype: int64


# Équilibrage du jeu de données

On équilibre la taille des label pour avoir un nombre équivalent de rating VRAI et FAUX.

In [4]:
#Fonction d'équilibrage du jeu de données
def balanceSample(X, size, classe):
  data = pd.DataFrame()
  for i in classe:
    x = X.loc[X['our rating'].isin([i])]
    if( x.shape[0] > size) :
      x_new =  pd.concat([x[x['our rating'] == c].sample(size, replace=False)for c in x['our rating'].unique()])
    elif (x.shape[0] < size) :
      if(x.shape[0]*2 >= size):
        x_new = pd.concat([x,pd.concat([x[x['our rating'] == c].sample((size-x.shape[0]), replace=False)for c in x['our rating'].unique()])], ignore_index = True)
      else :
        x_new = pd.concat([x,pd.concat([x[x['our rating'] == c].sample((size-x.shape[0]), replace=True)for c in x['our rating'].unique()])], ignore_index = True)
    else :
      x_new = x
    data = pd.concat([data,x_new], ignore_index = True)
  print(data['our rating'].value_counts())
  
  return data

In [5]:
#Équilibrage des données True/False
#Étant donné que l'on a 815 FALSE et 411 TRUE, on arrondis pour avoir 500 texte de chaque
balanced_concat = balanceSample(df_concat,500,["true","false"])

#Sauvegarde dans un fichier .csv d'un unique jeu de donnée traité qui va servir pour toutes les classification VRAI/FAUX
# balanced_concat.to_csv('./data_csv/balanced_concat.csv')
# df = pd.read_csv('./data_csv/balanced_concat.csv')

true     500
false    500
Name: our rating, dtype: int64
